#### Initial data exploration and prep for `main.py`

In [12]:
from dotenv import load_dotenv
import os
import requests
import time

In [13]:
load_dotenv()

True

PDF link to CTA API documentation: https://www.transitchicago.com/assets/1/6/cta_Train_Tracker_API_Developer_Guide_and_Documentation.pdf

* will use the _locations API_ to track multiple locations at once
* produces a list of in service trains w basic info
* 193 [double check] total CTA stops maybe one dict per line
    * line_name = {cta_train_stop_key_id: friendltrain_name}
* json response most important
    * ctatt: root element
    * tmst: time stamp
    * train: container element one per train in response
    * destNm: friendly destination description
    * nextStpId: next stop ID matcing GTFS
    * nextStaId: next station ID matching GTFS
    * nextStaNm: proper name of next station
* do another response request for massive list of parent stop IDs {statId:StatNm}
    * parent stop IDs are for the station overall
    * each direction at the station has it's own ID for additional detail not needed for this project
* create another dictionary that maps StatId to gio pin
* link to total number of stations per train line: https://rtams.org/services/cta/lines
* {parent_station_id: GPIO_PIN}

In [ ]:
CTA_API_KEY = os.getenv('CTA_API_KEY')

valid_rt_names = ["Red", "Blue", "Brn", "G", "Org", "P", "Pink", "Y"]


location_url = ''.join(['http://lapi.transitchicago.com/api/1.0/ttpositions.aspx?key=',
                            CTA_API_KEY, 
                            '&rt=', valid_rt_names[0],
                            '&rt=', valid_rt_names[1],
                            '&rt=', valid_rt_names[2],
                            '&rt=', valid_rt_names[3],
                            '&outputType=JSON'])


response = requests.get(location_url).json()

stop = 0
train = 0

all_nxt_sta_names = []
all_nxt_sta_dict= {}

rt_len = len(response['ctatt']['route'])
for train in range(rt_len):
        for stop in range(len(response['ctatt']['route'][train]['train'])):
                all_nxt_sta_dict[response['ctatt']['route'][train]['train'][stop]['nextStaId']] = response['ctatt']['route'][train]['train'][stop]['nextStaNm']
                all_nxt_sta_names.append(response['ctatt']['route'][train]['train'][stop]['nextStaNm'])
                stop+=1
        train+=1

print(all_nxt_sta_dict, '\n', all_nxt_sta_names)



{'41450': 'Chicago', '40540': 'Wilson', '40910': '63rd', '41490': 'Harrison', '40330': 'Grand', '40650': 'North/Clybourn', '40880': 'Thorndale', '41000': 'Cermak-Chinatown', '41090': 'Monroe', '41320': 'Belmont', '41220': 'Fullerton', '41170': 'Garfield', '40450': '95th/Dan Ryan', '40080': 'Sheridan', '41200': 'Argyle', '40890': "O'Hare", '41330': 'Montrose', '40060': 'Belmont', '40670': "Western (O'Hare Branch)", '40010': 'Austin', '40550': 'Irving Park', '40820': 'Rosemont', '40350': 'UIC-Halsted', '40750': "Harlem (O'Hare Branch)", '40220': 'Western (Forest Park Branch)', '40980': 'Harlem (Forest Park Branch)', '40920': 'Pulaski', '40590': 'Damen', '40180': 'Oak Park', '40810': 'Illinois Medical District', '40380': 'Clark/Lake', '41280': 'Jefferson Park', '41410': 'Chicago', '40070': 'Jackson', '40970': 'Cicero', '41340': 'LaSalle', '41010': 'Rockwell', '40710': 'Chicago', '41310': 'Paulina', '41210': 'Wellington', '40870': 'Francisco', '41290': 'Kimball', '41180': 'Kedzie', '40090'

In [52]:
# start building function for API call ...

def get_half_trns_lapi_call(is_first_half_rt_lst: bool) -> dict:

    CTA_API_KEY = os.getenv('CTA_API_KEY')

    rt_names = ["Red", "Blue", "Brn", "G", "Org", "P", "Pink", "Y"]

    
    if is_first_half_rt_lst==1: 
        location_url = ''.join(['http://lapi.transitchicago.com/api/1.0/ttpositions.aspx?key='
                                ,CTA_API_KEY
                                ,'&rt=', rt_names[0]
                                ,'&rt=', rt_names[1]
                                ,'&rt=', rt_names[2]    # api has max of four train lines at once
                                ,'&rt=', rt_names[3]
                                ,'&outputType=JSON'])

    else:
        location_url = ''.join(['http://lapi.transitchicago.com/api/1.0/ttpositions.aspx?key='
                                ,CTA_API_KEY
                                ,'&rt=', rt_names[-4]
                                ,'&rt=', rt_names[-3]
                                ,'&rt=', rt_names[-2]
                                ,'&rt=', rt_names[-1]
                                ,'&outputType=JSON'])
        

    response = requests.get(location_url).json()


    stop = 0
    train = 0
    rt_len = len(response['ctatt']['route'])
    all_nxt_sta_dict= {} # {stationId: station_name}


    for train in range(rt_len):
        
        for stop in range(len(response['ctatt']['route'][train]['train'])): # active stops on train line
                all_nxt_sta_dict[int(response['ctatt']['route'][train]['train'][stop]['nextStaId'])] = response['ctatt']['route'][train]['train'][stop]['nextStaNm']
                stop+=1

        train+=1


    return all_nxt_sta_dict

In [53]:
get_half_trns_lapi_call(is_first_half_rt_lst=False)

{40310: 'Western',
 41130: 'Halsted',
 40040: 'Quincy',
 40930: 'Midway',
 40960: 'Pulaski',
 40900: 'Howard',
 40400: 'Noyes',
 40740: 'Western',
 40170: 'Ashland',
 41030: 'Polk',
 40850: 'Harold Washington Library-State/Van Buren',
 41680: 'Oakton-Skokie'}

In [54]:
get_half_trns_lapi_call(is_first_half_rt_lst=True)

{41000: 'Cermak-Chinatown',
 40450: '95th/Dan Ryan',
 41490: 'Harrison',
 41420: 'Addison',
 41170: 'Garfield',
 41300: 'Loyola',
 40650: 'North/Clybourn',
 41220: 'Fullerton',
 41400: 'Roosevelt',
 40080: 'Sheridan',
 40760: 'Granville',
 41320: 'Belmont',
 41450: 'Chicago',
 40910: '63rd',
 40470: 'Racine',
 40010: 'Austin',
 40250: 'Kedzie-Homan',
 40670: "Western (O'Hare Branch)",
 40750: "Harlem (O'Hare Branch)",
 41330: 'Montrose',
 41410: 'Chicago',
 41240: 'Addison',
 40980: 'Harlem (Forest Park Branch)',
 40810: 'Illinois Medical District',
 40800: 'Sedgwick',
 40660: 'Armitage',
 40680: 'Adams/Wabash',
 40090: 'Damen',
 40360: 'Southport',
 40710: 'Chicago',
 41480: 'Western',
 41700: 'Washington/Wabash',
 40020: 'Harlem/Lake',
 40290: 'Ashland/63rd',
 40130: '51st',
 41710: 'Damen',
 41120: '35th-Bronzeville-IIT',
 40480: 'Cicero',
 40700: 'Laramie',
 40170: 'Ashland'}